<a href="https://colab.research.google.com/github/Konst-Andre25/-VK-/blob/main/Copy_of_BestStudyModelRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray=='2.2.0'
!pip install torch =='2.1.0+cu121'
!pip install torchvision=='0.16.0+cu121'
!pip install -U tensorboardx
!pip install "pydantic<2"
!pip install --upgrade google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 49.7 MB/s eta 0:00:00
ERROR: Invalid requirement: '==2.1.0+cu121'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.3
    Uninstalling pydantic-2.6.3:
      Successfully uninstalled pydantic-2.6.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [2]:

from functools import partial
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler

import torch.utils.data as data
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output

import numpy as np
from functools import partial
import pandas as pd

from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint as pp
import io

In [4]:
'''Настройка работы с Google Drive'''

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'cryps-414010-041e36d94a74.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

results = service.files().list(pageSize=10,
                               fields="nextPageToken, files(id, name, mimeType)").execute()

In [5]:
'''Разделим датасет на тестовую и обучающую выборки'''
def DataPreprocess(df, train_percent):
    scaler = StandardScaler()

    '''Преобразуем Dataset к типу float32 и выделим timeseries и таргет данные'''
    df = df.iloc[:, :].values.astype('float32')
    timeseries = df
    target = df[:, 0]

    '''Нормализуем данные для обчуения'''
    timeseries = scaler.fit_transform(timeseries)

    '''Разделим данные на train и тест'''
    train_size = int(len(df)*train_percent)
    X_train, X_test = timeseries[:train_size], timeseries[train_size:]
    y_train, y_test = target[:train_size], target[train_size:]

    return torch.tensor(X_train), torch.tensor(y_train), torch.tensor(X_test), torch.tensor(y_test)

In [6]:
'''Данная функция преобразует датасет с учётом скользящего окна для работы в формате временных рядов'''
def SlidingWindow(X, y, window):
    X_window, y_window = torch.ones(len(y) - window, window, X.shape[1]) * np.nan, torch.ones(len(y) - window, window) * np.nan
    for i in range(len(X)- window):
        X_window[i] = X[i:i+window, :]
        y_window[i] = y[i+1:i+window+1]
    return X_window, y_window

In [7]:
'''Модель нейронной сети на базе LSTM ячейки'''
class NN_LSTM(nn.Module):
    def __init__(self, metric_count=1, hidden_size=500, dropout = 0.5, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size=metric_count, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.drop = nn.Dropout(p = dropout)
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.drop(x)
        x = self.linear(x)
        return x

In [8]:
'''Модель нейронной сети на базе GRU ячейки'''
class NN_GRU(nn.Module):
    def __init__(self, metric_count=1, hidden_size=500, dropout = 0.5, num_layers=2):
        super().__init__()
        self.lstm = nn.GRU(input_size=metric_count, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.drop = nn.Dropout(p = dropout)
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.drop(x)
        x = self.linear(x)
        return x

In [9]:
'''Модель нейронной сети на базе GRU ячейки'''
class NN_RNN(nn.Module):
    def __init__(self, metric_count=1, hidden_size=500, dropout = 0.5, num_layers=2):
        super().__init__()
        self.lstm = nn.RNN(input_size=metric_count, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.drop = nn.Dropout(p = dropout)
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.drop(x)
        x = self.linear(x)
        return x

In [10]:
def load_data(window=3, train_percent=0.8, df=None):
        train_size = int(len(df)*train_percent)
        X_train, y_train, X_test, y_test = DataPreprocess(df = df, train_percent = train_percent)
        X_train, y_train = SlidingWindow(X_train, y_train, window)
        X_test, y_test = SlidingWindow(X_test, y_test, window)

        return (X_train.cuda(), y_train.cuda(), X_test.cuda(), y_test.cuda())

In [11]:
def train_cifar(config, df):
    train_percent=config['train_percent']
    metric_count=df.shape[1]
    window=config['window']
    num_epoch = config['epoch']

    net = NN_RNN(metric_count=metric_count, hidden_size=config['hidden'], dropout=config['drop'], num_layers=config['n_layer'])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.cuda()

    checkpoint = session.get_checkpoint()

    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        net.load_state_dict(checkpoint_state["net_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    X_train, y_train, X_test, y_test = load_data(window=window, train_percent=train_percent, df = df)
    train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=int(config['batch_size']))
    val_loader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=True, batch_size=int(config['batch_size']))


    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=config["lr"])

    for epoch in range(start_epoch, num_epoch):
        running_loss = 0.0
        epoch_steps = 0
        size = len(train_loader.dataset)
        for batch, (X_batch, y_batch) in enumerate(train_loader):

            X_batch,y_batch = X_batch, y_batch

            optimizer.zero_grad()

            y_pred = net(X_batch).view(-1, window)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if batch % 500 == 0:  # print every 100 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, batch + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

    # Validation loss
    val_loss = 0.0
    val_steps = 0

    for batch, (X_test, y_test) in enumerate(val_loader):
            with torch.no_grad():
                y_pred = net(X_test).view(-1, window)
                loss = criterion(y_pred, y_test).cpu()
                rmse_loss = np.sqrt(loss)
                val_loss += rmse_loss.cpu().numpy()
                val_steps += 1


    checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
    checkpoint = Checkpoint.from_dict(checkpoint_data)

    session.report(
            {"loss": val_loss/val_steps},
            checkpoint=checkpoint,
        )



In [12]:
def test_loss(model, window,train_percent, df, f):

        X_train, y_train, X_test, Y_test = load_data(window=window, train_percent=0.1, df = df)
        test_loader = data.DataLoader(data.TensorDataset(X_test, Y_test), shuffle=False, batch_size=1)
        criter = nn.MSELoss()
        val_loss = 0.0
        val_steps = 0
        R_out = 1
        ret = [R_out]
        trading_strategy = list()
        FinalValue = 1
        InitValue = 1

        for _, (X_test, y_test) in enumerate(test_loader):
            with torch.no_grad():
                y_pred = model(X_test).view(-1, window)
                loss = criter(y_pred, y_test).cpu()
                rmse_loss = np.sqrt(loss)
                val_loss += rmse_loss.cpu().numpy()
                val_steps += 1
                close = y_pred[-1,-1]
                open = y_pred[-1,-2]
                R_c = ((1-f)*close-(1+f)*open)/open
                R_out = R_out*(1+R_c)
                ret.append(R_out)
                # trading_decision = trading_algo(R_c)
                # trading_strategy.append(trading_decision)

        ROI = (R_out-1)/1
        return val_loss/val_steps,  ret, Y_test


In [13]:
def main( num_s = 50, max_num_epoch= 10):
    # Выгрузка данных в датафрейм
    metrics= ['PriceUSD', 'TxTfrValMedUSD', 'SplyExpFut10yr', 'TxTfrValMedNtv', 'VtyDayRet180d',
    'VtyDayRet30d']
    metric_count = len(metrics)
    df = pd.read_csv(r'Data/ETH_PriceUSD.csv')[metrics].dropna()

    config = {
                "window": tune.choice(np.arange(1, 5)),
                "train_percent": tune.loguniform(0.8, 0.85),
                "hidden": tune.choice([2 ** i for i in range(8, 10)]),
                "drop": tune.loguniform(1e-1, 3e-1),
                "lr": tune.loguniform(1e-4, 1e-1),
                "n_layer": tune.choice([1]),
                "batch_size": tune.choice([2, 4, 8, 11, 16]),
                "epoch":max_num_epoch
                }


    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epoch,
        grace_period=1,
        reduction_factor=2,
    )

    result = tune.run(
        partial(train_cifar, df=df),
        resources_per_trial={"cpu": 2, "gpu": 1},
        config=config,
        num_samples=num_s,
        scheduler=scheduler,
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")


    best_trained_model = NN_RNN(metric_count=metric_count, hidden_size=best_trial.config['hidden'], dropout=best_trial.config['drop'], num_layers=best_trial.config['n_layer'])

    best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
    best_checkpoint_data = best_checkpoint.to_dict()
    best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])

    torch.save(best_trained_model.state_dict(), r"Data/BestModels/BestModel.pt")

    with open(r'Data/BestModels/BestConfig.txt', "w") as f:
        f.writelines(f"{param} = {best_trial.config[param]}\n" for param in best_trial.config)

    folder_id = '1Ksmrr0Q7f4Dstz_oE3XHu6_jeOx2BuEI'

    model_filename = 'BestModel.pt'
    config_filename = 'BestConfig.txt'

    model_filepath = 'Data/BestModels/BestModel.pt'
    file_metadata = {
                    'name': model_filename,
                    'parents': [folder_id]
                }
    media = MediaFileUpload(model_filepath, resumable=True)
    r = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    pp.pprint(r)

    config_filepath = 'Data/BestModels/BestConfig.txt'
    file_metadata = {
                    'name': config_filename,
                    'parents': [folder_id]
                }
    media = MediaFileUpload(config_filepath, resumable=True)
    r = service.files().create(body=file_metadata, media_body=media, fields='id', ocrLanguage = 'en').execute()
    pp.pprint(r)



In [15]:
if __name__ == "__main__":
    main(num_s=2, max_num_epoch=5)


(func pid=2018) [3,   501] loss: 127767.646
(func pid=2120) [1,     1] loss: 153339.922
(func pid=2120) [2,     1] loss: 45412.586
(func pid=2120) [3,     1] loss: 161748.531
(func pid=2120) [4,     1] loss: 10958.486
(func pid=2120) [5,     1] loss: 78899.805


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_80ab1_00000,2024-03-13_05-35-39,True,,d387958309684ab6a7e41397597e29ca,738eac4684f5,1,313.133,172.28.0.12,2120,True,5.0908,5.0908,5.0908,1710308139,0,,1,80ab1_00000,0.0117235
train_cifar_80ab1_00001,2024-03-13_05-35-48,True,,5ae37086476f4a2da58c201f8552b9f0,738eac4684f5,1,1668.53,172.28.0.12,2253,True,5.08463,5.08463,5.08463,1710308148,0,,1,80ab1_00001,0.0034318


(func pid=2253) [1,     1] loss: 2401125.500
(func pid=2253) [2,     1] loss: 3109616.000
(func pid=2253) [3,     1] loss: 2007300.375
(func pid=2253) [4,     1] loss: 579350.875
(func pid=2253) [5,     1] loss: 767461.375


2024-03-13 05:35:49,092	INFO tune.py:762 -- Total run time: 19.60 seconds (19.45 seconds for the tuning loop).


Best trial config: {'window': 2, 'train_percent': 0.8040883544510221, 'hidden': 512, 'drop': 0.17393007885348166, 'lr': 0.04976662892030535, 'n_layer': 1, 'batch_size': 8, 'epoch': 5}
Best trial final validation loss: 313.1325689951579
{'id': '1YADMieSSkx0qx1B7mq1rK672UFDkJubx'}
{'id': '1NGGLYPuNoNjjMrVLCp_T6wp4uoexY2g-'}
